In [1]:
import pandas as pd
import os
import requests
import json
import time

os.chdir('/home/anovax/open_data/edrsr_data')

In [6]:
decisions = pd.read_csv('most_recent/data_full.csv')

In [12]:
judges = decisions.judge.unique()

In [15]:
# function to check a match of downloaded declaration

def check_and_filter(objects, judge_ln, judge_fn, judge_pn, position='суддя'):
    
    cut_treshold = len('суддя')
    
    filtered_objects = []
    
    for dec in objects:

        dec_last_name = dec['infocard']['last_name']
        dec_fn = dec['infocard']['first_name'][0]
        dec_pn = dec['infocard']['patronymic'][0]
        
        init_pos_name = dec['infocard']['position']
        
        if 'суддя' in init_pos_name.lower() and len(init_pos_name) > cut_treshold:
            dec_position = dec['infocard']['position'][:cut_treshold].lower()
        else:
            
            dec_position = dec['infocard']['position'].lower()



        if (dec_last_name, dec_fn, dec_pn, dec_position) == (judge_ln, judge_fn, judge_pn, position):
            filtered_objects.append(dec)
            
    return filtered_objects

In [70]:
# script to download declarations from pages and save them into json file

declarations_by_judge = {}

for i, judge in enumerate(judges, start=1):

    declarations = []

    print(f'Searching for judge {judge}... - #{i}: Left: {judges.shape[0]-i}')
    
    # catching different patterns in names like Lastname N.N. or N.N. Lastname
    if '.' in judge:
        inter_split = judge.split('.')
        
        if len(inter_split[0]) >= 2:
            judge_name_split = (inter_split[0] + '. ' + inter_split[1] + '.').replace('  ', ' ').split()
        
        else:
            judge_name_split = (inter_split[-1] + ' ' + inter_split[0] + '. ' + inter_split[1] + '.').replace('  ', ' ').split()
        
    else:
        inter_split = judge.split()
        judge_name_split = [word[0]+'.' if idx == 1 or idx==2 else word  for idx, word in enumerate(inter_split)]
        
    

    judge_last_name = judge_name_split[0]
    judge_fn = judge_name_split[1][0]
    judge_pn = judge_name_split[2][0]
    position = 'суддя'

    url = f'https://declarations.com.ua/search?q={judge_last_name}+суддя&deepsearch=on&format=opendata&page=1'

    print("Fetching page #%s" % 1)

    r = requests.get(url).json()

    objects = r['results']['object_list']

    filtered_objects = check_and_filter(objects, judge_last_name, judge_fn, judge_pn)

    print(f'Found {len(filtered_objects)} objects')

    declarations += filtered_objects        

    for page in range(2, r["results"]["paginator"]["num_pages"] + 1):
        time.sleep(0.5)
        print("Fetching page #%s" % page)

        subr = requests.get(
            "https://declarations.com.ua/search?q=%s+суддя&deepsearch=on&format=opendata&page=%s" % (judge_last_name, page)).json()


        objects = subr['results']['object_list']

        filtered_objects = check_and_filter(objects, judge_last_name, judge_fn, judge_pn)

        declarations += filtered_objects


    print(f'Got {len(declarations)} declarations')

    declarations_by_judge[judge] = declarations


    print(f'Successfuly saved to dict {len(declarations_by_judge[judge])} declarations')



with open('declarations_by_judge_v2.json', 'w') as f:
    json.dump(declarations_by_judge, f, indent=2, ensure_ascii=False)

Searching for judge Бобрик Г.Й.... - #1: Left: 724
Fetching page #1
Found 8 objects
Got 8 declarations
Successfuly saved to dict 8 declarations
Searching for judge Йосипчук О.С.... - #2: Left: 723
Fetching page #1
Found 13 objects
Got 13 declarations
Successfuly saved to dict 13 declarations
Searching for judge Пригара Л.І.... - #3: Left: 722
Fetching page #1
Found 7 objects
Got 7 declarations
Successfuly saved to dict 7 declarations
Searching for judge Ремецькі О.Ф.... - #4: Left: 721
Fetching page #1
Found 5 objects
Got 5 declarations
Successfuly saved to dict 5 declarations
Searching for judge Васьковський О.В.... - #5: Left: 720
Fetching page #1
Found 4 objects
Got 4 declarations
Successfuly saved to dict 4 declarations
Searching for judge Андрейчук Л.В.... - #6: Left: 719
Fetching page #1
Found 6 objects
Got 6 declarations
Successfuly saved to dict 6 declarations
Searching for judge Ушак І.Г.... - #7: Left: 718
Fetching page #1
Found 6 objects
Got 6 declarations
Successfuly saved 

Fetching page #2
Got 36 declarations
Successfuly saved to dict 36 declarations
Searching for judge Разіна Т.І.... - #50: Left: 675
Fetching page #1
Found 27 objects
Got 27 declarations
Successfuly saved to dict 27 declarations
Searching for judge Тищенко А.І.... - #51: Left: 674
Fetching page #1
Found 26 objects
Fetching page #2
Fetching page #3
Got 30 declarations
Successfuly saved to dict 30 declarations
Searching for judge Сотніков С.В.... - #52: Left: 673
Fetching page #1
Found 19 objects
Got 19 declarations
Successfuly saved to dict 19 declarations
Searching for judge Мартюк А.І.... - #53: Left: 672
Fetching page #1
Found 30 objects
Fetching page #2
Got 38 declarations
Successfuly saved to dict 38 declarations
Searching for judge Яковлєв М.Л.... - #54: Left: 671
Fetching page #1
Found 18 objects
Fetching page #2
Got 22 declarations
Successfuly saved to dict 22 declarations
Searching for judge Остапенко О.М.... - #55: Left: 670
Fetching page #1
Found 22 objects
Fetching page #2
Fet

Found 6 objects
Got 6 declarations
Successfuly saved to dict 6 declarations
Searching for judge Привалов А.І.... - #100: Left: 625
Fetching page #1
Found 7 objects
Got 7 declarations
Successfuly saved to dict 7 declarations
Searching for judge Ващенко Т.М.... - #101: Left: 624
Fetching page #1
Found 4 objects
Fetching page #2
Got 6 declarations
Successfuly saved to dict 6 declarations
Searching for judge Паламар П.І.... - #102: Left: 623
Fetching page #1
Found 8 objects
Got 8 declarations
Successfuly saved to dict 8 declarations
Searching for judge Котков О.В.... - #103: Left: 622
Fetching page #1
Found 6 objects
Got 6 declarations
Successfuly saved to dict 6 declarations
Searching for judge Трофименко Т.Ю.... - #104: Left: 621
Fetching page #1
Found 8 objects
Got 8 declarations
Successfuly saved to dict 8 declarations
Searching for judge Сівакова В.В.... - #105: Left: 620
Fetching page #1
Found 8 objects
Got 8 declarations
Successfuly saved to dict 8 declarations
Searching for judge Г

Found 7 objects
Got 7 declarations
Successfuly saved to dict 7 declarations
Searching for judge Церковна Н.Ф.... - #151: Left: 574
Fetching page #1
Found 6 objects
Got 6 declarations
Successfuly saved to dict 6 declarations
Searching for judge Політика Н.А.... - #152: Left: 573
Fetching page #1
Found 9 objects
Got 9 declarations
Successfuly saved to dict 9 declarations
Searching for judge Андрійчук О.В.... - #153: Left: 572
Fetching page #1
Found 10 objects
Got 10 declarations
Successfuly saved to dict 10 declarations
Searching for judge Горплюк А.М.... - #154: Left: 571
Fetching page #1
Found 8 objects
Got 8 declarations
Successfuly saved to dict 8 declarations
Searching for judge Пашкевич І.О.... - #155: Left: 570
Fetching page #1
Found 7 objects
Got 7 declarations
Successfuly saved to dict 7 declarations
Searching for judge Торчинюк В.Г.... - #156: Left: 569
Fetching page #1
Found 13 objects
Got 13 declarations
Successfuly saved to dict 13 declarations
Searching for judge Романюк Ю.

Found 5 objects
Fetching page #2
Got 6 declarations
Successfuly saved to dict 6 declarations
Searching for judge Чонгова С.І.... - #206: Left: 519
Fetching page #1
Found 4 objects
Got 4 declarations
Successfuly saved to dict 4 declarations
Searching for judge Лутак Т.В.... - #207: Left: 518
Fetching page #1
Found 4 objects
Got 4 declarations
Successfuly saved to dict 4 declarations
Searching for judge Мальована Л.Я.... - #208: Left: 517
Fetching page #1
Found 4 objects
Got 4 declarations
Successfuly saved to dict 4 declarations
Searching for judge Саванчук С.О.... - #209: Left: 516
Fetching page #1
Found 7 objects
Got 7 declarations
Successfuly saved to dict 7 declarations
Searching for judge Христенко О.О.... - #210: Left: 515
Fetching page #1
Found 5 objects
Got 5 declarations
Successfuly saved to dict 5 declarations
Searching for judge Горбасенко П.В.... - #211: Left: 514
Fetching page #1
Found 9 objects
Got 9 declarations
Successfuly saved to dict 9 declarations
Searching for judge

...

Found 7 objects
Got 7 declarations
Successfuly saved to dict 7 declarations
Searching for judge Смірнов О.Г.... - #518: Left: 207
Fetching page #1
Found 8 objects
Fetching page #2
Got 11 declarations
Successfuly saved to dict 11 declarations
Searching for judge Сушко Л.М.... - #519: Left: 206
Fetching page #1
Found 2 objects
Fetching page #2
Fetching page #3
Got 9 declarations
Successfuly saved to dict 9 declarations
Searching for judge Мірошниченко М.В.... - #520: Left: 205
Fetching page #1
Found 1 objects
Fetching page #2
Got 4 declarations
Successfuly saved to dict 4 declarations
Searching for judge Проскуряков К.В.... - #521: Left: 204
Fetching page #1
Found 7 objects
Got 7 declarations
Successfuly saved to dict 7 declarations
Searching for judge Зінченко Н.Г.... - #522: Left: 203
Fetching page #1
Found 6 objects
Fetching page #2
Fetching page #3
Got 9 declarations
Successfuly saved to dict 9 declarations
Searching for judge Корсун В.Л.... - #523: Left: 202
Fetching page #1
Found 8

Found 4 objects
Got 4 declarations
Successfuly saved to dict 4 declarations
Searching for judge Дучал Н.М.... - #568: Left: 157
Fetching page #1
Found 7 objects
Got 7 declarations
Successfuly saved to dict 7 declarations
Searching for judge Зубченко І.В.... - #569: Left: 156
Fetching page #1
Found 7 objects
Got 7 declarations
Successfuly saved to dict 7 declarations
Searching for judge Радіонова О.О.... - #570: Left: 155
Fetching page #1
Found 6 objects
Got 6 declarations
Successfuly saved to dict 6 declarations
Searching for judge Марченко О.А.... - #571: Left: 154
Fetching page #1
Found 5 objects
Fetching page #2
Fetching page #3
Fetching page #4
Got 6 declarations
Successfuly saved to dict 6 declarations
Searching for judge Попков Д.О.... - #572: Left: 153
Fetching page #1
Found 4 objects
Got 4 declarations
Successfuly saved to dict 4 declarations
Searching for judge М'ясищев А.М.... - #573: Left: 152
Fetching page #1
Found 4 objects
Got 4 declarations
Successfuly saved to dict 4 de

Fetching page #7
Fetching page #8
Fetching page #9
Got 15 declarations
Successfuly saved to dict 15 declarations
Searching for judge Давченко Т.М.... - #616: Left: 109
Fetching page #1
Found 7 objects
Got 7 declarations
Successfuly saved to dict 7 declarations
Searching for judge Міщенко В.І.... - #617: Left: 108
Fetching page #1
Found 3 objects
Fetching page #2
Fetching page #3
Fetching page #4
Got 4 declarations
Successfuly saved to dict 4 declarations
Searching for judge Олейняш Е.М.... - #618: Left: 107
Fetching page #1
Found 9 objects
Got 9 declarations
Successfuly saved to dict 9 declarations
Searching for judge Кабакова В.Г.... - #619: Left: 106
Fetching page #1
Found 6 objects
Got 6 declarations
Successfuly saved to dict 6 declarations
Searching for judge Вавренюк Л.С.... - #620: Left: 105
Fetching page #1
Found 2 objects
Got 2 declarations
Successfuly saved to dict 2 declarations
Searching for judge Коваленко Н.М.... - #621: Left: 104
Fetching page #1
Found 0 objects
Fetching 

Found 4 objects
Fetching page #2
Got 5 declarations
Successfuly saved to dict 5 declarations
Searching for judge Резніченко Олена Юріївна... - #666: Left: 59
Fetching page #1
Found 5 objects
Got 5 declarations
Successfuly saved to dict 5 declarations
Searching for judge Спиридонова Надія Олександрівна... - #667: Left: 58
Fetching page #1
Found 9 objects
Fetching page #2
Got 9 declarations
Successfuly saved to dict 9 declarations
Searching for judge Соп`яненко Оксана Юріївна... - #668: Left: 57
Fetching page #1
Found 0 objects
Got 0 declarations
Successfuly saved to dict 0 declarations
Searching for judge Коваленко Олександр Вікторович... - #669: Left: 56
Fetching page #1
Found 0 objects
Fetching page #2
Fetching page #3
Fetching page #4
Fetching page #5
Fetching page #6
Fetching page #7
Fetching page #8
Fetching page #9
Fetching page #10
Got 8 declarations
Successfuly saved to dict 8 declarations
Searching for judge Левченко Павло Іванович... - #670: Left: 55
Fetching page #1
Found 5 o

Found 7 objects
Fetching page #2
Got 8 declarations
Successfuly saved to dict 8 declarations
Searching for judge Огороднік Костянтин Михайлович... - #715: Left: 10
Fetching page #1
Found 14 objects
Fetching page #2
Got 18 declarations
Successfuly saved to dict 18 declarations
Searching for judge Демидюк Оксана Олександрівна... - #716: Left: 9
Fetching page #1
Found 11 objects
Got 11 declarations
Successfuly saved to dict 11 declarations
Searching for judge Розізнана Інна Вікторівна... - #717: Left: 8
Fetching page #1
Found 14 objects
Got 14 declarations
Successfuly saved to dict 14 declarations
Searching for judge Філіпова Тетяна Леонідівна... - #718: Left: 7
Fetching page #1
Found 12 objects
Got 12 declarations
Successfuly saved to dict 12 declarations
Searching for judge Мельник Олег Володимирович... - #719: Left: 6
Fetching page #1
Found 0 objects
Fetching page #2
Fetching page #3
Fetching page #4
Fetching page #5
Fetching page #6
Fetching page #7
Fetching page #8
Fetching page #9
F

In [2]:
# loading json file

with open('declarations_by_judge_v2.json') as f:
    dec_json = json.load(f)

In [75]:
judges_full_names = [' '.join((value[0]['infocard']['last_name'],
                     value[0]['infocard']['first_name'],
                     value[0]['infocard']['patronymic']))
                     for key, value in declarations_by_judge.items() if value != []]

In [77]:
judges_series = pd.Series(judges_full_names)

judges_series.name = 'judge_full_name'

judges_series.to_csv('judges_full_names.csv', index=False)

## Extracting information from ring.org.ua

In [129]:
# to display intermediate dec status info

def display_rel_dec_status(objects):
    if objects != []:

        for item in objects:
            dec_type = item['intro']['declaration_type']
            general = item['general']

            year_from = item['intro']['declaration_year_from']
            year_to = item['intro']['declaration_year_to']


            print(f'Declaration type: {dec_type}', general['name'], general['last_name'], general['patronymic'],
                  'from ' + year_from + ' to ' + year_to)

In [141]:
def display_garnahata_status(objects):
    
    if objects != []:
        for item in objects:
            
            print('MATCH! ----- ' + item['owner'] + ': ', item['asset'])

In [139]:
def get_ring_info(judges_names, search_filter):
    
    declarations_by_judge = {}


    for i, judge in enumerate(judges_names, start=1):

        declarations = []
        
        if search_filter == 'relations':
            datasource = 'vkks'
            display_function = display_rel_dec_status
            
        elif search_filter == 'garnahata':
            datasource = 'garnahata_in_ua'
            display_function = display_garnahata_status
            

        print(f'Searching for judge {judge}... - #{i}: Left: {len(judges_names)-i}')

        url = f'https://ring.org.ua/search?q={judge}&datasources={datasource}&format=json&page=1'

        print("Fetching page #%s" % 1)
        
        time.sleep(0.5)

        r = requests.get(url).json()

        objects = r['search_results']['object_list']

        print(f'Found {len(objects)} objects')
        
        
        display_function(objects) if display_function is not None else None
                

        declarations += objects        

        for page in range(2, r["search_results"]["paginator"]["num_pages"] + 1):
            time.sleep(0.5)
            print("Fetching page #%s" % page)

            subr = requests.get(
                f'https://ring.org.ua/search?q={judge}&datasources={datasource}&format=json&page={page}').json()


            objects = subr['search_results']['object_list']

            display_function(objects) if display_function is not None else None

            declarations += objects


        print(f'Got total {len(declarations)} declarations')

        declarations_by_judge[judge] = declarations


        print(f'Successfuly saved to dict {len(declarations_by_judge[judge])} declarations')
        
    return declarations_by_judge

In [128]:
relations = get_ring_info(judges_full_names, 'relations')

Searching for judge Бобрик Ганна Йосипівна... - #1: Left: 493
Fetching page #1
Found 1 objects
Declaration type: родинних зв’язків судді Ганна Бобрик Йосипівна from 2013 to 2017
Got total 1 declarations
Successfuly saved to dict 1 declarations
Searching for judge Йосипчук Олександр Семенович... - #2: Left: 492
Fetching page #1
Found 4 objects
Declaration type: родинних зв’язків кандидата на посаду судді Олександр Йосипчук Семенович from 2013 to 2017
Declaration type: родинних зв’язків кандидата на посаду судді Олександр Йосипчук Семенович from 2012 to 2017
Declaration type: родинних зв’язків судді Олександр Йосипчук Семенович from 2013 to 2017
Declaration type: родинних зв’язків судді Олександр Йосипчук Семенович from 2013 to 2017
Got total 4 declarations
Successfuly saved to dict 4 declarations
Searching for judge Пригара Леся Іванівна... - #3: Left: 491
Fetching page #1
Found 2 objects
Declaration type: родинних зв’язків судді Леся Пригара Іванівна from 2014 to 2018
Declaration type:

Found 7 objects
Declaration type: родинних зв’язків судді В. Федорова О. Null from 2014 to 2018
Declaration type: родинних зв’язків судді В. Федорова О. Null from 2014 to 2018
Declaration type: родинних зв’язків судді Яна Карабань Анатоліївна from 2014 to 2018
Declaration type: родинних зв’язків кандидата на посаду судді В. Федорова О. Null from 2013 to 2017
Declaration type: родинних зв’язків судді Владислав Сулім Володимирович from 2014 to 2018
Declaration type: родинних зв’язків судді Владислав Сулім Володимирович from 2014 to 2018
Declaration type: родинних зв’язків судді Владислав Сулім Володимирович from 2013 to 2017
Got total 7 declarations
Successfuly saved to dict 7 declarations
Searching for judge Разіна Тетяна Іванівна... - #28: Left: 466
Fetching page #1
Found 2 objects
Declaration type: родинних зв’язків кандидата на посаду судді Тетяна Разіна Іванівна from 2013 to 2017
Declaration type: родинних зв’язків судді Тетяна Разіна Іванівна from 2014 to 2018
Got total 2 declarati

Found 1 objects
Declaration type: родинних зв’язків судді Андрій Василишин Романович from 2014 to 2018
Got total 1 declarations
Successfuly saved to dict 1 declarations
Searching for judge Пінчук Валерій Іванович... - #56: Left: 438
Fetching page #1
Found 1 objects
Declaration type: родинних зв’язків судді Анатолій Стрижеус Миколайович from 2014 to 2018
Got total 1 declarations
Successfuly saved to dict 1 declarations
Searching for judge Маринченко Ярослав Вікторович... - #57: Left: 437
Fetching page #1
Found 1 objects
Declaration type: родинних зв’язків судді Ярослав Маринченко Вікторович from 2014 to 2018
Got total 1 declarations
Successfuly saved to dict 1 declarations
Searching for judge Мандриченко Олександр Володимирович... - #58: Left: 436
Fetching page #1
Found 1 objects
Declaration type: родинних зв’язків судді Олександр Мандриченко Володимирович from 2014 to 2018
Got total 1 declarations
Successfuly saved to dict 1 declarations
Searching for judge Шкурдова Людмила Миколаївна.

Got total 1 declarations
Successfuly saved to dict 1 declarations
Searching for judge Чеберяк Павло Петрович... - #89: Left: 405
Fetching page #1
Found 1 objects
Declaration type: родинних зв’язків судді Павло Чеберяк Петрович from 2014 to 2018
Got total 1 declarations
Successfuly saved to dict 1 declarations
Searching for judge Картавцева Юлія Валеріївна... - #90: Left: 404
Fetching page #1
Found 2 objects
Declaration type: родинних зв’язків кандидата на посаду судді Юлія Картавцева Валеріївна from 2013 to 2017
Declaration type: родинних зв’язків судді Юлія Картавцева Валеріївна from 2014 to 2018
Got total 2 declarations
Successfuly saved to dict 2 declarations
Searching for judge Кирилюк Тетяна Юріївна... - #91: Left: 403
Fetching page #1
Found 1 objects
Declaration type: родинних зв’язків судді Тетяна Кирилюк Юріївна from 2014 to 2018
Got total 1 declarations
Successfuly saved to dict 1 declarations
Searching for judge Ковтун Сергій Анатолійович... - #92: Left: 402
Fetching page #1


Found 2 objects
Declaration type: родинних зв’язків кандидата на посаду судді Віталій Войтюк Романович from 2013 to 2018
Declaration type: родинних зв’язків судді Віталій Войтюк Романович from 2014 to 2018
Got total 2 declarations
Successfuly saved to dict 2 declarations
Searching for judge Бережнюк Віта Вячеславівна... - #117: Left: 377
Fetching page #1
Found 2 objects
Declaration type: родинних зв’язків судді Віта Бережнюк Вячеславівна from 2014 to 2018
Declaration type: родинних зв’язків судді Віта Бережнюк Вячеславівна from 2014 to 2018
Got total 2 declarations
Successfuly saved to dict 2 declarations
Searching for judge Марач Віктор Васильович... - #118: Left: 376
Fetching page #1
Found 1 objects
Declaration type: родинних зв’язків судді Віктор Марач Васильович from 2014 to 2018
Got total 1 declarations
Successfuly saved to dict 1 declarations
Searching for judge Церковна Наталія Федорівна... - #119: Left: 375
Fetching page #1
Found 1 objects
Declaration type: родинних зв’язків су

...

Found 1 objects
Declaration type: родинних зв’язків судді Іван Шумський Петрович from 2014 to 2018
Got total 1 declarations
Successfuly saved to dict 1 declarations
Searching for judge Боровець Ярослав Ярославович... - #363: Left: 131
Fetching page #1
Found 2 objects
Declaration type: родинних зв’язків судді Ярослав Боровець Ярославович from 2014 to 2018
Declaration type: родинних зв’язків судді Ярослав Боровець Ярославович from 2013 to 2017
Got total 2 declarations
Successfuly saved to dict 2 declarations
Searching for judge Стопник Сергій Григорович... - #364: Left: 130
Fetching page #1
Found 2 objects
Declaration type: родинних зв’язків судді Сергій Стопник Григорович from 2014 to 2018
Declaration type: родинних зв’язків кандидата на посаду судді Сергій Стопник Григорович from 2013 to 2017
Got total 2 declarations
Successfuly saved to dict 2 declarations
Searching for judge Бурда Надія Миколаївна... - #365: Left: 129
Fetching page #1
Found 1 objects
Declaration type: родинних зв’язк

Found 11 objects
Declaration type: родинних зв’язків судді Марія Демидова Олександрівна from 2013 to 2017
Declaration type: родинних зв’язків судді Алла Демидова Маркіянівна from 2014 to 2018
Declaration type: родинних зв’язків судді Алла Демидова Маркіянівна from 2013 to 2017
Declaration type: родинних зв’язків судді Марія Демидова Олександрівна from 2013 to 2017
Declaration type: родинних зв’язків судді Марія Демидова Олександрівна from 2014 to 2018
Declaration type: родинних зв’язків кандидата на посаду судді Марія Демидова Олександрівна from 2013 to 2018
Declaration type: родинних зв’язків судді Марія Демидова Олександрівна from 2013 to 2017
Declaration type: родинних зв’язків судді Марія Демидова Олександрівна from 2014 to 2018
Declaration type: родинних зв’язків кандидата на посаду судді Сергій Бурлаков Юрійович from 2013 to 2017
Declaration type: родинних зв’язків судді Владислав Демидов Олександрович from 2013 to 2017
Declaration type: родинних зв’язків судді Владислав Демидов 

Found 0 objects
Got total 0 declarations
Successfuly saved to dict 0 declarations
Searching for judge Ткаченко Олександр Васильович... - #421: Left: 73
Fetching page #1
Found 6 objects
Declaration type: родинних зв’язків судді Олександр Ткаченко Васильович from 2014 to 2018
Declaration type: родинних зв’язків судді Олександр Ткаченко Васильович from 2013 to 2017
Declaration type: родинних зв’язків кандидата на посаду судді Олександр Ткаченко Васильович from  to 
Declaration type: родинних зв’язків судді Олександр Ткаченко Васильович from 2013 to 2017
Declaration type: родинних зв’язків судді Олександр Ткаченко Васильович from 2014 to 2018
Declaration type: родинних зв’язків судді Олександр Ткаченко Васильович from 2014 to 2018
Got total 6 declarations
Successfuly saved to dict 6 declarations
Searching for judge Давченко Тетяна Миколаївна... - #422: Left: 72
Fetching page #1
Found 1 objects
Declaration type: родинних зв’язків судді Тетяна Давченко Миколаївна from 2014 to 2018
Got total 

Found 0 objects
Got total 0 declarations
Successfuly saved to dict 0 declarations
Searching for judge Масловський Сергій Володимирович... - #447: Left: 47
Fetching page #1
Found 1 objects
Declaration type: родинних зв’язків кандидата на посаду судді Руслан Мельник Іванович from 2013 to 2017
Got total 1 declarations
Successfuly saved to dict 1 declarations
Searching for judge Старкова Галина Микитівна... - #448: Left: 46
Fetching page #1
Found 0 objects
Got total 0 declarations
Successfuly saved to dict 0 declarations
Searching for judge Лісовицький Євген Артурович... - #449: Left: 45
Fetching page #1
Found 1 objects
Declaration type: родинних зв’язків судді Євген Лісовицький Артурович from 2014 to 2018
Got total 1 declarations
Successfuly saved to dict 1 declarations
Searching for judge Зюбанова Наталія Миколаївна... - #450: Left: 44
Fetching page #1
Found 1 objects
Declaration type: родинних зв’язків судді Наталія Зюбанова Миколаївна from 2014 to 2018
Got total 1 declarations
Successf

Found 2 objects
Declaration type: родинних зв’язків судді Ірина Ткаченко Вікторівна from 2014 to 2018
Declaration type: родинних зв’язків судді Ніна Ткаченко Григорівна from 2014 to 2018
Got total 2 declarations
Successfuly saved to dict 2 declarations
Searching for judge Суховий Валерій Григорович... - #476: Left: 18
Fetching page #1
Found 0 objects
Got total 0 declarations
Successfuly saved to dict 0 declarations
Searching for judge Пєсков В'ячеслав Геннадійович... - #477: Left: 17
Fetching page #1
Found 1 objects
Declaration type: родинних зв’язків судді В'ячеслав Пєсков Геннадійович from 2013 to 2017
Got total 1 declarations
Successfuly saved to dict 1 declarations
Searching for judge Погребняк Володимир Яковлевич... - #478: Left: 16
Fetching page #1
Found 2 objects
Declaration type: родинних зв’язків судді Володимир Погребняк Яковлевич from 2014 to 2018
Declaration type: родинних зв’язків кандидата на посаду судді Володимир Погребняк Яковлевич from 2014 to 2018
Got total 2 declara

In [140]:
elite_real_estate = get_ring_info(judges_full_names, 'garnahata')

Searching for judge Бобрик Ганна Йосипівна... - #1: Left: 493
Fetching page #1
Found 0 objects
Got total 0 declarations
Successfuly saved to dict 0 declarations
Searching for judge Йосипчук Олександр Семенович... - #2: Left: 492
Fetching page #1
Found 0 objects
Got total 0 declarations
Successfuly saved to dict 0 declarations
Searching for judge Пригара Леся Іванівна... - #3: Left: 491
Fetching page #1
Found 0 objects
Got total 0 declarations
Successfuly saved to dict 0 declarations
Searching for judge Ремецькі Оксана Федорівна... - #4: Left: 490
Fetching page #1
Found 0 objects
Got total 0 declarations
Successfuly saved to dict 0 declarations
Searching for judge Васьковський Олег Вікторович... - #5: Left: 489
Fetching page #1
Found 0 objects
Got total 0 declarations
Successfuly saved to dict 0 declarations
Searching for judge Андрейчук Любомир Вікторович... - #6: Left: 488
Fetching page #1
Found 0 objects
Got total 0 declarations
Successfuly saved to dict 0 declarations
Searching for 

Found 0 objects
Got total 0 declarations
Successfuly saved to dict 0 declarations
Searching for judge Скрипка Ірина Михайлівна... - #51: Left: 443
Fetching page #1
Found 0 objects
Got total 0 declarations
Successfuly saved to dict 0 declarations
Searching for judge Пантелієнко Віталій Олегович... - #52: Left: 442
Fetching page #1
Found 0 objects
Got total 0 declarations
Successfuly saved to dict 0 declarations
Searching for judge Сотніков Сергій Вікторович... - #53: Left: 441
Fetching page #1
Found 0 objects
Got total 0 declarations
Successfuly saved to dict 0 declarations
Searching for judge Шаптала Євген Юрійович... - #54: Left: 440
Fetching page #1
Found 0 objects
Got total 0 declarations
Successfuly saved to dict 0 declarations
Searching for judge Василишин Андрій Романович... - #55: Left: 439
Fetching page #1
Found 0 objects
Got total 0 declarations
Successfuly saved to dict 0 declarations
Searching for judge Пінчук Валерій Іванович... - #56: Left: 438
Fetching page #1
Found 0 obj

Found 0 objects
Got total 0 declarations
Successfuly saved to dict 0 declarations
Searching for judge Чебикіна Світлана Олександрівна... - #100: Left: 394
Fetching page #1
Found 0 objects
Got total 0 declarations
Successfuly saved to dict 0 declarations
Searching for judge Лиськов Максим Олександрович... - #101: Left: 393
Fetching page #1
Found 0 objects
Got total 0 declarations
Successfuly saved to dict 0 declarations
Searching for judge Усатенко Інга Вікторівна... - #102: Left: 392
Fetching page #1
Found 0 objects
Got total 0 declarations
Successfuly saved to dict 0 declarations
Searching for judge Любченко Марія Олександрівна... - #103: Left: 391
Fetching page #1
Found 0 objects
Got total 0 declarations
Successfuly saved to dict 0 declarations
Searching for judge Літвінова Марина Євгенівна... - #104: Left: 390
Fetching page #1
Found 0 objects
Got total 0 declarations
Successfuly saved to dict 0 declarations
Searching for judge Блажівська Оксана Євгенівна... - #105: Left: 389
Fetchin

Found 0 objects
Got total 0 declarations
Successfuly saved to dict 0 declarations
Searching for judge Хабазня Юрій Андрійович... - #150: Left: 344
Fetching page #1
Found 0 objects
Got total 0 declarations
Successfuly saved to dict 0 declarations
Searching for judge Боровик Сергій Сергійович... - #151: Left: 343
Fetching page #1
Found 0 objects
Got total 0 declarations
Successfuly saved to dict 0 declarations
Searching for judge Бацуца Віталій Михайлович... - #152: Left: 342
Fetching page #1
Found 0 objects
Got total 0 declarations
Successfuly saved to dict 0 declarations
Searching for judge Грабець Світлана Юріївна... - #153: Left: 341
Fetching page #1
Found 0 objects
Got total 0 declarations
Successfuly saved to dict 0 declarations
Searching for judge Лопатін Антон Володимирович... - #154: Left: 340
Fetching page #1
Found 0 objects
Got total 0 declarations
Successfuly saved to dict 0 declarations
Searching for judge Рябцева Ольга Олексіївна... - #155: Left: 339
Fetching page #1
Found 

Found 0 objects
Got total 0 declarations
Successfuly saved to dict 0 declarations
Searching for judge Байбак Олег Іванович... - #201: Left: 293
Fetching page #1
Found 0 objects
Got total 0 declarations
Successfuly saved to dict 0 declarations
Searching for judge Буракова Алла Миколаївна... - #202: Left: 292
Fetching page #1
Found 0 objects
Got total 0 declarations
Successfuly saved to dict 0 declarations
Searching for judge Лаврова Любов Степанівна... - #203: Left: 291
Fetching page #1
Found 0 objects
Got total 0 declarations
Successfuly saved to dict 0 declarations
Searching for judge Новікова Наталія Анатоліївна... - #204: Left: 290
Fetching page #1
Found 0 objects
Got total 0 declarations
Successfuly saved to dict 0 declarations
Searching for judge Бринцев Олексій Васильович... - #205: Left: 289
Fetching page #1
Found 1 objects
Бринцев Олексій Васильович:  квартира; Заг.пл.: 223.6, Житл.пл.: 94.1
Got total 1 declarations
Successfuly saved to dict 1 declarations
Searching for judge Ш

Found 0 objects
Got total 0 declarations
Successfuly saved to dict 0 declarations
Searching for judge Шніт Альона Василівна... - #250: Left: 244
Fetching page #1
Found 0 objects
Got total 0 declarations
Successfuly saved to dict 0 declarations
Searching for judge Кравець Світлана Георгіївна... - #251: Left: 243
Fetching page #1
Found 0 objects
Got total 0 declarations
Successfuly saved to dict 0 declarations
Searching for judge Маріщенко Лариса Олександрівна... - #252: Left: 242
Fetching page #1
Found 0 objects
Got total 0 declarations
Successfuly saved to dict 0 declarations
Searching for judge Лозинська Ірина Василівна... - #253: Left: 241
Fetching page #1
Found 0 objects
Got total 0 declarations
Successfuly saved to dict 0 declarations
Searching for judge Давидюк Валерій Кіндратович... - #254: Left: 240
Fetching page #1
Found 0 objects
Got total 0 declarations
Successfuly saved to dict 0 declarations
Searching for judge Костриця Олег Олександрович... - #255: Left: 239
Fetching page 

Found 0 objects
Got total 0 declarations
Successfuly saved to dict 0 declarations
Searching for judge Найфлейш Володимир Давидович... - #301: Left: 193
Fetching page #1
Found 0 objects
Got total 0 declarations
Successfuly saved to dict 0 declarations
Searching for judge Грабован Лілія Іванівна... - #302: Left: 192
Fetching page #1
Found 0 objects
Got total 0 declarations
Successfuly saved to dict 0 declarations
Searching for judge Морозюк Андрій Ярославович... - #303: Left: 191
Fetching page #1
Found 0 objects
Got total 0 declarations
Successfuly saved to dict 0 declarations
Searching for judge Кітаєва Світлана Богданівна... - #304: Left: 190
Fetching page #1
Found 0 objects
Got total 0 declarations
Successfuly saved to dict 0 declarations
Searching for judge Козак Ірина Богданівна... - #305: Left: 189
Fetching page #1
Found 0 objects
Got total 0 declarations
Successfuly saved to dict 0 declarations
Searching for judge Мороз Наталія Валеріївна... - #306: Left: 188
Fetching page #1
Foun

Found 0 objects
Got total 0 declarations
Successfuly saved to dict 0 declarations
Searching for judge Литвинова Вікторія Володимирівна... - #349: Left: 145
Fetching page #1
Found 0 objects
Got total 0 declarations
Successfuly saved to dict 0 declarations
Searching for judge Павленко Наталія Анатоліївна... - #350: Left: 144
Fetching page #1
Found 0 objects
Got total 0 declarations
Successfuly saved to dict 0 declarations
Searching for judge Пригуза Павло Дмитрович... - #351: Left: 143
Fetching page #1
Found 0 objects
Got total 0 declarations
Successfuly saved to dict 0 declarations
Searching for judge Немченко Любов Миколаївна... - #352: Left: 142
Fetching page #1
Found 0 objects
Got total 0 declarations
Successfuly saved to dict 0 declarations
Searching for judge Стадник Марія Семенівна... - #353: Left: 141
Fetching page #1
Found 0 objects
Got total 0 declarations
Successfuly saved to dict 0 declarations
Searching for judge Чопко Юрій Олегович... - #354: Left: 140
Fetching page #1
Foun

Found 0 objects
Got total 0 declarations
Successfuly saved to dict 0 declarations
Searching for judge Склярук Ольга Ігорівна... - #400: Left: 94
Fetching page #1
Found 0 objects
Got total 0 declarations
Successfuly saved to dict 0 declarations
Searching for judge Мартюхіна Наталя Олександрівна... - #401: Left: 93
Fetching page #1
Found 0 objects
Got total 0 declarations
Successfuly saved to dict 0 declarations
Searching for judge Будко Наталя Владиславівна... - #402: Left: 92
Fetching page #1
Found 0 objects
Got total 0 declarations
Successfuly saved to dict 0 declarations
Searching for judge Чернота Людмила Федорівна... - #403: Left: 91
Fetching page #1
Found 0 objects
Got total 0 declarations
Successfuly saved to dict 0 declarations
Searching for judge Стойка Оксана Володимирівна... - #404: Left: 90
Fetching page #1
Found 0 objects
Got total 0 declarations
Successfuly saved to dict 0 declarations
Searching for judge Геза Таісія Дмитрівна... - #405: Left: 89
Fetching page #1
Found 0 o

Found 0 objects
Got total 0 declarations
Successfuly saved to dict 0 declarations
Searching for judge Зюбанова Наталія Миколаївна... - #450: Left: 44
Fetching page #1
Found 0 objects
Got total 0 declarations
Successfuly saved to dict 0 declarations
Searching for judge Яресько Богдан Володимирович... - #451: Left: 43
Fetching page #1
Found 0 objects
Got total 0 declarations
Successfuly saved to dict 0 declarations
Searching for judge Шеліхіна Ріта Михайлівна... - #452: Left: 42
Fetching page #1
Found 0 objects
Got total 0 declarations
Successfuly saved to dict 0 declarations
Searching for judge Драгнєвіч Олена Володимирівна... - #453: Left: 41
Fetching page #1
Found 0 objects
Got total 0 declarations
Successfuly saved to dict 0 declarations
Searching for judge Фонова Олена Сергіївна... - #454: Left: 40
Fetching page #1
Found 0 objects
Got total 0 declarations
Successfuly saved to dict 0 declarations
Searching for judge Палей Олена Сергіївна... - #455: Left: 39
Fetching page #1
Found 0 o

### Actually aggregated information can be accessed only within each separate declaration link. With &format=json instead of format=opendata

In [93]:
def get_dec_aggr(json_file):
    
    declarations_by_judge = {}
    
    judges_names = list(json_file.keys())


    for i, judge in enumerate(judges_names, start=1):

        declarations = []
        
        judges_items = json_file[judge]

        print(f'Searching for judge {judge}... - #{i}: Left: {len(judges_names)-i}')
        
        counter = 0
        
        for declaration in judges_items:
            
            counter += 1
            
            url = declaration['infocard']['url'] + '?format=json'
            
            time.sleep(0.5)

            aggr_dec_json = requests.get(url).json()
                
            declarations += [aggr_dec_json]
            
            if_aggregated = 'aggregated' in aggr_dec_json['declaration'].keys()
            
            print(f'---- #{counter} Aggregated: {if_aggregated}')



        print(f'GOT TOTAL {len(declarations)} declarations')

        declarations_by_judge[judge] = declarations
        
    return declarations_by_judge

In [94]:
def get_one_aggr(judge, json_file):
    
    declarations_by_judge = {}

    declarations = []

    judges_items = json_file[judge]

    print(f'Searching for judge {judge}')

    counter = 0

    for declaration in judges_items:

        counter += 1

        url = declaration['infocard']['url'] + '?format=json'

        time.sleep(0.5)

        aggr_dec_json = requests.get(url).json()

        declarations += [aggr_dec_json]

        if_aggregated = 'aggregated' in aggr_dec_json['declaration'].keys()

        print(f'---- #{counter} Aggregated: {if_aggregated}')
        
        
#         if counter == 2:
        
#             return declarations
        
    return declarations

#     print(f'GOT TOTAL {len(declarations)} declarations')

#     declarations_by_judge[judge] = declarations
        
#     return declarations_by_judge

In [95]:
aggregated_by_judge = get_dec_aggr(declarations_by_judge)

with open('aggregations_by_judge.json', 'w') as f:
    json.dump(aggregated_by_judge, f, indent=2, ensure_ascii=False)

Searching for judge Бобрик Г.Й.... - #1: Left: 724
---- #1 Aggregated: True
---- #2 Aggregated: True
---- #3 Aggregated: False
---- #4 Aggregated: True
---- #5 Aggregated: True
---- #6 Aggregated: True
---- #7 Aggregated: True
---- #8 Aggregated: False
GOT TOTAL 8 declarations
Searching for judge Йосипчук О.С.... - #2: Left: 723
---- #1 Aggregated: True
---- #2 Aggregated: True
---- #3 Aggregated: True
---- #4 Aggregated: True
---- #5 Aggregated: False
---- #6 Aggregated: True
---- #7 Aggregated: True
---- #8 Aggregated: True
---- #9 Aggregated: True
---- #10 Aggregated: True
---- #11 Aggregated: True
---- #12 Aggregated: False
---- #13 Aggregated: False
GOT TOTAL 13 declarations
Searching for judge Пригара Л.І.... - #3: Left: 722
---- #1 Aggregated: True
---- #2 Aggregated: True
---- #3 Aggregated: True
---- #4 Aggregated: False
---- #5 Aggregated: True
---- #6 Aggregated: True
---- #7 Aggregated: False
GOT TOTAL 7 declarations
Searching for judge Ремецькі О.Ф.... - #4: Left: 721
----

---- #15 Aggregated: False
---- #16 Aggregated: True
---- #17 Aggregated: False
---- #18 Aggregated: False
---- #19 Aggregated: False
GOT TOTAL 19 declarations
Searching for judge Данко Леся Семенівна... - #24: Left: 701
---- #1 Aggregated: True
---- #2 Aggregated: True
---- #3 Aggregated: True
---- #4 Aggregated: True
---- #5 Aggregated: False
---- #6 Aggregated: True
---- #7 Aggregated: False
---- #8 Aggregated: False
---- #9 Aggregated: False
GOT TOTAL 9 declarations
Searching for judge Дубник Оксана Петрівна... - #25: Left: 700
---- #1 Aggregated: False
---- #2 Aggregated: False
---- #3 Aggregated: False
---- #4 Aggregated: False
---- #5 Aggregated: True
---- #6 Aggregated: False
---- #7 Aggregated: True
---- #8 Aggregated: True
---- #9 Aggregated: False
---- #10 Aggregated: True
---- #11 Aggregated: False
---- #12 Aggregated: False
---- #13 Aggregated: False
---- #14 Aggregated: False
---- #15 Aggregated: False
---- #16 Aggregated: False
GOT TOTAL 16 declarations
Searching for jud

---- #1 Aggregated: False
---- #2 Aggregated: False
---- #3 Aggregated: False
---- #4 Aggregated: False
---- #5 Aggregated: False
---- #6 Aggregated: False
---- #7 Aggregated: False
---- #8 Aggregated: False
---- #9 Aggregated: False
---- #10 Aggregated: False
---- #11 Aggregated: False
---- #12 Aggregated: False
---- #13 Aggregated: True
---- #14 Aggregated: True
---- #15 Aggregated: False
---- #16 Aggregated: True
---- #17 Aggregated: True
---- #18 Aggregated: False
---- #19 Aggregated: False
---- #20 Aggregated: False
GOT TOTAL 20 declarations
Searching for judge Хрипун О.О.... - #41: Left: 684
GOT TOTAL 0 declarations
Searching for judge Дикунська С.Я.... - #42: Left: 683
---- #1 Aggregated: False
---- #2 Aggregated: False
---- #3 Aggregated: False
---- #4 Aggregated: False
---- #5 Aggregated: False
---- #6 Aggregated: True
---- #7 Aggregated: False
---- #8 Aggregated: False
---- #9 Aggregated: False
---- #10 Aggregated: False
---- #11 Aggregated: False
---- #12 Aggregated: False
-

---- #18 Aggregated: False
---- #19 Aggregated: False
---- #20 Aggregated: False
---- #21 Aggregated: False
---- #22 Aggregated: False
---- #23 Aggregated: False
---- #24 Aggregated: False
---- #25 Aggregated: False
---- #26 Aggregated: True
---- #27 Aggregated: True
---- #28 Aggregated: True
---- #29 Aggregated: False
---- #30 Aggregated: False
GOT TOTAL 30 declarations
Searching for judge Сотніков С.В.... - #52: Left: 673
---- #1 Aggregated: False
---- #2 Aggregated: False
---- #3 Aggregated: False
---- #4 Aggregated: False
---- #5 Aggregated: False
---- #6 Aggregated: True
---- #7 Aggregated: False
---- #8 Aggregated: False
---- #9 Aggregated: False
---- #10 Aggregated: False
---- #11 Aggregated: False
---- #12 Aggregated: False
---- #13 Aggregated: False
---- #14 Aggregated: True
---- #15 Aggregated: True
---- #16 Aggregated: True
---- #17 Aggregated: True
---- #18 Aggregated: False
---- #19 Aggregated: False
GOT TOTAL 19 declarations
Searching for judge Мартюк А.І.... - #53: Left:

---- #9 Aggregated: False
---- #10 Aggregated: False
---- #11 Aggregated: False
---- #12 Aggregated: False
---- #13 Aggregated: False
---- #14 Aggregated: False
---- #15 Aggregated: False
---- #16 Aggregated: False
---- #17 Aggregated: False
---- #18 Aggregated: False
---- #19 Aggregated: False
---- #20 Aggregated: False
---- #21 Aggregated: False
---- #22 Aggregated: False
---- #23 Aggregated: False
---- #24 Aggregated: False
---- #25 Aggregated: False
---- #26 Aggregated: False
---- #27 Aggregated: False
---- #28 Aggregated: True
---- #29 Aggregated: True
---- #30 Aggregated: True
---- #31 Aggregated: True
---- #32 Aggregated: True
---- #33 Aggregated: False
---- #34 Aggregated: False
---- #35 Aggregated: False
GOT TOTAL 35 declarations
Searching for judge Пантелієнко В.О.... - #63: Left: 662
---- #1 Aggregated: True
---- #2 Aggregated: False
---- #3 Aggregated: False
---- #4 Aggregated: False
---- #5 Aggregated: True
---- #6 Aggregated: False
---- #7 Aggregated: False
---- #8 Aggreg

...

---- #1 Aggregated: True
---- #2 Aggregated: True
---- #3 Aggregated: False
---- #4 Aggregated: True
---- #5 Aggregated: True
---- #6 Aggregated: True
---- #7 Aggregated: False
GOT TOTAL 7 declarations
Searching for judge Лісовицький Є.А.... - #653: Left: 72
---- #1 Aggregated: True
---- #2 Aggregated: True
---- #3 Aggregated: True
---- #4 Aggregated: True
---- #5 Aggregated: False
GOT TOTAL 5 declarations
Searching for judge Зюбанова Н.М.... - #654: Left: 71
---- #1 Aggregated: True
---- #2 Aggregated: True
---- #3 Aggregated: True
---- #4 Aggregated: True
---- #5 Aggregated: False
GOT TOTAL 5 declarations
Searching for judge Яресько Б.В.... - #655: Left: 70
---- #1 Aggregated: True
---- #2 Aggregated: True
---- #3 Aggregated: True
---- #4 Aggregated: True
---- #5 Aggregated: True
---- #6 Aggregated: True
---- #7 Aggregated: True
---- #8 Aggregated: True
---- #9 Aggregated: False
GOT TOTAL 9 declarations
Searching for judge Шеліхіна Р.М.... - #656: Left: 69
---- #1 Aggregated: True
--

---- #1 Aggregated: False
---- #2 Aggregated: False
---- #3 Aggregated: False
---- #4 Aggregated: False
---- #5 Aggregated: False
---- #6 Aggregated: False
---- #7 Aggregated: False
---- #8 Aggregated: False
---- #9 Aggregated: True
---- #10 Aggregated: True
---- #11 Aggregated: False
---- #12 Aggregated: False
---- #13 Aggregated: False
---- #14 Aggregated: False
---- #15 Aggregated: False
---- #16 Aggregated: False
---- #17 Aggregated: False
---- #18 Aggregated: False
---- #19 Aggregated: False
---- #20 Aggregated: False
---- #21 Aggregated: False
---- #22 Aggregated: True
---- #23 Aggregated: True
---- #24 Aggregated: True
---- #25 Aggregated: False
---- #26 Aggregated: False
---- #27 Aggregated: False
GOT TOTAL 27 declarations
Searching for judge Катеринчук Л.Й.... - #678: Left: 47
---- #1 Aggregated: False
---- #2 Aggregated: False
---- #3 Aggregated: False
---- #4 Aggregated: False
---- #5 Aggregated: False
---- #6 Aggregated: False
---- #7 Aggregated: False
---- #8 Aggregated: F

---- #14 Aggregated: False
---- #15 Aggregated: False
---- #16 Aggregated: False
---- #17 Aggregated: False
---- #18 Aggregated: False
---- #19 Aggregated: False
---- #20 Aggregated: False
---- #21 Aggregated: False
---- #22 Aggregated: True
---- #23 Aggregated: True
---- #24 Aggregated: False
---- #25 Aggregated: False
GOT TOTAL 25 declarations
Searching for judge Дроботова Т.Б.... - #688: Left: 37
---- #1 Aggregated: False
---- #2 Aggregated: False
---- #3 Aggregated: False
---- #4 Aggregated: False
---- #5 Aggregated: False
---- #6 Aggregated: False
---- #7 Aggregated: True
---- #8 Aggregated: False
---- #9 Aggregated: False
---- #10 Aggregated: False
---- #11 Aggregated: False
---- #12 Aggregated: False
---- #13 Aggregated: False
---- #14 Aggregated: False
---- #15 Aggregated: False
---- #16 Aggregated: True
---- #17 Aggregated: False
---- #18 Aggregated: False
---- #19 Aggregated: False
---- #20 Aggregated: False
---- #21 Aggregated: False
---- #22 Aggregated: False
---- #23 Aggre

---- #11 Aggregated: False
---- #12 Aggregated: False
---- #13 Aggregated: False
---- #14 Aggregated: False
---- #15 Aggregated: False
---- #16 Aggregated: False
---- #17 Aggregated: True
---- #18 Aggregated: False
---- #19 Aggregated: False
---- #20 Aggregated: False
---- #21 Aggregated: True
---- #22 Aggregated: True
---- #23 Aggregated: False
---- #24 Aggregated: False
---- #25 Aggregated: False
---- #26 Aggregated: False
GOT TOTAL 26 declarations
Searching for judge Уркевич Віталій Юрійович... - #699: Left: 26
---- #1 Aggregated: False
---- #2 Aggregated: False
---- #3 Aggregated: False
---- #4 Aggregated: False
---- #5 Aggregated: False
---- #6 Aggregated: False
---- #7 Aggregated: False
---- #8 Aggregated: False
---- #9 Aggregated: False
---- #10 Aggregated: False
---- #11 Aggregated: False
---- #12 Aggregated: False
---- #13 Aggregated: False
---- #14 Aggregated: False
---- #15 Aggregated: False
---- #16 Aggregated: False
---- #17 Aggregated: False
---- #18 Aggregated: False
---

---- #1 Aggregated: False
---- #2 Aggregated: True
---- #3 Aggregated: False
---- #4 Aggregated: True
---- #5 Aggregated: True
---- #6 Aggregated: False
---- #7 Aggregated: False
---- #8 Aggregated: True
---- #9 Aggregated: True
---- #10 Aggregated: True
---- #11 Aggregated: True
---- #12 Aggregated: True
---- #13 Aggregated: True
---- #14 Aggregated: False
GOT TOTAL 14 declarations
Searching for judge Грязнов Василь Володимирович... - #720: Left: 5
---- #1 Aggregated: True
---- #2 Aggregated: False
---- #3 Aggregated: True
---- #4 Aggregated: False
---- #5 Aggregated: True
---- #6 Aggregated: True
---- #7 Aggregated: False
---- #8 Aggregated: False
GOT TOTAL 8 declarations
Searching for judge Чернота Людмила Федорівна... - #721: Left: 4
---- #1 Aggregated: True
---- #2 Aggregated: True
---- #3 Aggregated: True
---- #4 Aggregated: False
---- #5 Aggregated: True
---- #6 Aggregated: False
GOT TOTAL 6 declarations
Searching for judge Радіонова Олена Олександрівна... - #722: Left: 3
---- #